In [1]:
import pandas as pd
from hestia.similarity import *
from hestia.partition import ccpart
from hestia import HestiaGenerator
from hestia.clustering import _connected_components_clustering
from tqdm import tqdm


In [2]:
df = pd.read_csv("plumber.csv", sep=',')
df.head()

,seq,smiles
0,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,O=C(O)CCCCCN1C(=O)N(CCCCCC(=O)O)[C@H](Cc2ccccc...
1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,O=C1N(C/C=C/c2cn[nH]c2)[C@H](Cc2ccccc2)[C@H](O...
2,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,O=C1N(C/C=C/c2cn[nH]c2)[C@H](Cc2ccccc2)[C@H](O...
3,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,O=C1N(CCCCCCO)[C@H](Cc2ccccc2)[C@H](O)[C@@H](O...
4,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,O=C1N(CCCCCO)[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)...


In [3]:
df1_field = 'seq'
df2_field = 'smiles'

sim_fun_ent1 = sequence_similarity_mmseqs
sim_fun_ent2 = molecular_similarity

sim_args_ent1 = {
    "field_name": "seq",
    "threshold": 0.3,
    "verbose": 3
}
sim_args_ent2={
    "field_name": "smiles",
    "fingerprint": "ecfp",
    "radius": 2,
    "threshold": 0.3,
    "verbose": 3,
    "bits": 1024
}

In [4]:
unique_df1 = df.drop_duplicates(df1_field).reset_index(drop=True)
df1_to_df = df.groupby(df1_field).apply(lambda g: g.index.to_numpy())

sim_df_1 = sim_fun_ent1(df_query=unique_df1, **sim_args_ent1)
train, test, clusters = ccpart(
    df=unique_df1, sim_df=sim_df_1, threshold=0.3, verbose=True
)


/var/folders/11/g_ww0v454tvd_q2jybl8p2180000gn/T/ipykernel_38739/2417994603.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df1_to_df = df.groupby(df1_field).apply(lambda g: g.index.to_numpy())


Calculating pairwise alignments using MMSeqs2 algorithm with prefilter...
prefilter -s 6 hestia_tmp_1740058128.2587361/db_query hestia_tmp_1740058128.2587361/db_target hestia_tmp_1740058128.2587361/pref -v 3 

MMseqs Version:           	15-6f452
Substitution matrix       	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix  	aa:VTML80.out,nucl:nucleotide.out
Sensitivity               	6
k-mer length              	0
Target search mode        	0
k-score                   	seq:2147483647,prof:2147483647
Alphabet size             	aa:21,nucl:5
Max sequence length       	65535
Max results per query     	300
Split database            	0
Split mode                	2
Split memory limit        	0
Coverage threshold        	0
Coverage mode             	0
Compositional bias        	1
Compositional bias        	1
Diagonal scoring          	true
Exact k-mer matching      	0
Mask residues             	1
Mask residues probability 	0.9
Mask lower case residues  	0
Minimum diagonal score    	1

In [5]:
train_indices, test_indices = [], []
for indx in test:
    indcs = df1_to_df[indx]
    test_indices.extend(indcs)
for indx in train:
    indcs = df1_to_df[indx]
    train_indices.extend(indcs)

test_df, train_df = df.iloc[test_indices].reset_index(), df.iloc[train_indices].reset_index()

u_test = test_df.drop_duplicates(df2_field).reset_index()
u_train = train_df.drop_duplicates(df2_field).reset_index()

u_test_to_df = test_df.groupby(df2_field).apply(lambda g: g.index.to_numpy())
# u_train_to_df = u_train.groupby(df2_field).apply(lambda g: g.index.to_numpy())

unique_test_mols, unique_train_mols = u_test.smiles, u_train.smiles
print(len(unique_test_mols)/1e6, len(unique_train_mols)/1e6)



/var/folders/11/g_ww0v454tvd_q2jybl8p2180000gn/T/ipykernel_38739/2543338883.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  indcs = df1_to_df[indx]
/var/folders/11/g_ww0v454tvd_q2jybl8p2180000gn/T/ipykernel_38739/2543338883.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  indcs = df1_to_df[indx]


0.1498 0.793841


/var/folders/11/g_ww0v454tvd_q2jybl8p2180000gn/T/ipykernel_38739/2543338883.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  u_test_to_df = test_df.groupby(df2_field).apply(lambda g: g.index.to_numpy())


In [6]:
try:
    from rdkit import Chem
    from rdkit.Chem import rdFingerprintGenerator, rdMolDescriptors
    from rdkit.DataStructs import (
        BulkTanimotoSimilarity, BulkDiceSimilarity,
        BulkSokalSimilarity, BulkRogotGoldbergSimilarity,
        BulkCosineSimilarity)
    from rdkit import RDLogger
    from rdkit import rdBase

    def disable_rdkit_log():
        """Disable all rdkit logs."""
        for log_level in RDLogger._levels:
            rdBase.DisableLog(log_level)

    disable_rdkit_log()

except ModuleNotFoundError:
    raise ImportError("This function requires RDKit to be installed.")

radius, bits = 2, 1024
fpgen = rdFingerprintGenerator.GetMorganGenerator(
    radius=radius, fpSize=bits
)
sim_function = 'tanimoto'

def _get_fp(smile: str):
    mol = Chem.MolFromSmiles(smile, sanitize=True)

    if mol is None:
        print(f"SMILES: `{smile}` could not be processed. Will be substituted by `{smile[1:-1]}`")
        return _get_fp(smile[1:-1])

    fp = fpgen.GetFingerprint(mol)
    return fp

def _parallel_fps(mols: List[str], mssg: str) -> list:
    fps = []
    jobs = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        for mol in mols:
            job = executor.submit(_get_fp, mol)
            jobs.append(job)
        pbar = tqdm(jobs, desc=mssg, unit_scale=True,
                    mininterval=0.5, maxinterval=2)
        for job in pbar:
            if job.exception() is not None:
                raise RuntimeError(job.exception())
            result = job.result()
            fps.append(result)

    pbar.close()
    return fps

test_fps = _parallel_fps(unique_test_mols, "Test mols")

Test mols: 100%|██████████| 150k/150k [01:27<00:00, 1.70kit/s] 


In [15]:
from itertools import islice
from rdkit.DataStructs import ConvertToNumpyArray
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

def compare(mol, test_fps):
    fp = _get_fp(mol)
    sim = BulkTanimotoSimilarity(fp, test_fps)
    return sim

all_results = set()
threshold = sim_args_ent2['threshold']
# chunk_size = int(1e3)
# train_chunks = batched(unique_train_mols, n=chunk_size)
# pbar = tqdm(train_chunks, unit_scale=True, total=(len(unique_train_mols)//chunk_size)+1)
# with ThreadPoolExecutor(max_workers=10) as executor:
#     for chunk in pbar:
#         jobs = []
#         for mol in chunk:
#             job = executor.submit(compare, mol, test_fps)
#             jobs.append(job)

#         for job in jobs:
#             if job.exception() is not None:
#                 raise RuntimeError(job.exception())

#             out = job.result()
#             out = np.array(out)
#             out_u = np.argwhere(out > threshold)
#             all_results.update([i[0] for i in out_u.tolist()])
#             pbar.set_description(f"Exclude: {len(all_results):,} / {len(unique_test_mols):,}")
#             if len(all_results) == len(unique_test_mols):
#                 print("Saturation")
#                 break
#         if len(all_results) == len(unique_test_mols):
#             print("Saturation")
#             break
test_size = len(unique_test_mols)
pbar = tqdm(unique_train_mols, unit_scale=True)
import copy
tmp_fps = copy.deepcopy(test_fps)
prev_len = len(tmp_fps)
tmp_u_test = copy.deepcopy(unique_test_mols)

for idx, mol in enumerate(pbar):
    fp = _get_fp(mol)
    out = BulkTanimotoSimilarity(fp, tmp_fps)
    out = np.array(out)
    removed = [f for mni_idx, f in enumerate(tmp_u_test) if out[mni_idx] >= threshold]
    tmp_fps = [f for mni_idx, f in enumerate(tmp_fps) if out[mni_idx] < threshold]
    tmp_u_test = [f for mni_idx, f in enumerate(tmp_u_test) if out[mni_idx] < threshold]

    if len(tmp_fps) < prev_len:
        out_u = np.argwhere(out > threshold)
        all_results.update(removed)
    if idx % 100 == 0:
        pbar.set_description(f"Include: {len(tmp_fps):,} / {test_size:,}")
        if len(tmp_fps) == 0:
            print("Saturation")
            break

Include: 6,174 / 149,800:  18%|█▊        | 140k/794k [1:01:15<1:27:00, 125it/s]    

SMILES: `Cc1cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[OH-]` could not be processed. Will be substituted by `c1cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[OH-`
SMILES: `c1cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[OH-` could not be processed. Will be substituted by `1cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[OH`
SMILES: `1cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[OH` could not be processed. Will be substituted by `cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[O`
SMILES: `cc(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[O` could not be processed. Will be substituted by `c(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[`
SMILES: `c(F)cc(C)c1C(=O)c1sc2cc([Na+])ccc2c1-c1ccc(OCCN2CC(C(=O)[O-])C2)cc1.[Na+].[` could not be processed. Will be substi

Include: 5,872 / 149,800:  19%|█▉        | 151k/794k [1:02:36<56:53, 188it/s]   

SMILES: `c1ccc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)cc1` could not be processed. Will be substituted by `1ccc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)cc`
SMILES: `1ccc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)cc` could not be processed. Will be substituted by `ccc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)c`
SMILES: `ccc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)c` could not be processed. Will be substituted by `cc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)`
SMILES: `cc(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2)` could not be processed. Will be substituted by `c(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2`
SMILES: `c(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC2` could not be processed. Will be substituted by `(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC`
SMILES: `(CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)CC` could not be processed. Will be substituted by `CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)C`
SMILES: `CN2CCN(c3ccc4nc(C5=Nc6ccccc6[Na]5)oc4c3)C` could not be proce

Include: 3,543 / 149,800:  32%|███▏      | 255k/794k [1:09:48<33:33, 268it/s]   

...

Include: 1,508 / 149,800:  72%|███████▏  | 575k/794k [1:19:58<05:28, 668it/s]

SMILES: `CC(C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCCCO2` could not be processed. Will be substituted by `C(C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCCCO`
SMILES: `C(C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCCCO` could not be processed. Will be substituted by `(C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCCC`
SMILES: `(C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCCC` could not be processed. Will be substituted by `C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCC`
SMILES: `C)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CCC` could not be processed. Will be substituted by `)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CC`
SMILES: `)[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1CC` could not be processed. Will be substituted by `[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1C`
SMILES: `[N-]C(=CC=[Na+])c1ccc2c(c1)C(=O)Nc1cccc(n1)-c1nncn1C` could not be processed. Will be substituted by `N-]C(=CC=[Na+]

Include: 1,218 / 149,800:  79%|███████▉  | 630k/794k [1:21:12<03:24, 805it/s]

SMILES: `CNS(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)c1F` could not be processed. Will be substituted by `NS(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)c1`
SMILES: `NS(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)c1` could not be processed. Will be substituted by `S(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)c`
SMILES: `S(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)c` could not be processed. Will be substituted by `(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)`
SMILES: `(=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F)` could not be processed. Will be substituted by `=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F`
SMILES: `=O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F)c2F` could not be processed. Will be substituted by `O)(=O)[N-](->[Na+])c1nccc(Cc2cc(C(N)=O)c(Nc3ccc(C4CC4)cc3F)c(F

Include: 837 / 149,800: 100%|██████████| 794k/794k [1:24:48<00:00, 156it/s]    


In [16]:
# loose_indcs = np.zeros(len(test_df))
# for i in all_results:
#     loose_indcs[u_test_to_df[i]] = 1
# strict_indcs = loose_indcs < 1
loose_test = test_df[test_df.smiles.isin(all_results)]
strict_test = test_df[~test_df.smiles.isin(all_results)]
print(len(loose_test) / len(test_df), len(strict_test)/len(test_df))

0.9950084046519962 0.004991595348003789
